In [1]:
import pandas as pd
import numpy as np
import re
import os
import time
import matplotlib.pyplot as plt
from glob import glob
import sliced
import random
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sliced import SlicedAverageVarianceEstimation
from xgboost import XGBClassifier
# from numba import njit
from numpy.linalg import pinv
from tqdm import tqdm

In [7]:
def mahalanobis_transformation(feature_df, flag = "Train", feature_mean = np.empty((1,1)), sigma_pinv_sqrt = np.empty((1,1))):
    # whiten the data
    n = feature_df.shape[0]
    p = feature_df.shape[1]
    ones = np.ones((n, 1)) # n * 1
    if flag == "Train":
        # aviod np.mean in numba cannot use globals()
        feature_mean = np.empty((1,p))
        for i in range(p):
            feature_mean[0,i] = np.mean(feature_df[:,i], axis = 0)
        feature_centered = feature_df - np.dot(ones, feature_mean)
        # get cov matrix # p * p
        cov = np.dot(feature_centered.T, feature_centered) / n
        # u, s, vh = np.linalg.svd(cov) # all real numbers
        s, u = np.linalg.eigh(cov) # all real numbers
        # theshold for pinv: 1e-15
        s_pinv = np.linalg.pinv(np.diag(s))
        s_pinv_sqrt = np.zeros(p)
        for i, item in enumerate(np.diag(s_pinv)):
            if item > 0:
                s_pinv_sqrt[i] = np.sqrt(item)
        sigma_pinv_sqrt = np.dot(np.dot(u, np.diag(s_pinv_sqrt)), u.T)
        feature_df_whitened = np.dot(feature_centered, sigma_pinv_sqrt)
    else:
        if feature_mean is None or sigma_pinv_sqrt is None:
            raise ValueError("Need to assign feature mean vector and Sigma!")
        if feature_mean is not None and sigma_pinv_sqrt is not None:
            feature_centered = feature_df - np.dot(ones, feature_mean)
            feature_df_whitened = np.dot(feature_centered, sigma_pinv_sqrt)
    return feature_df_whitened, feature_mean, sigma_pinv_sqrt

def get_cov_mat(feature_df):
    n = feature_df.shape[0]
    p = feature_df.shape[1]
    ones = np.ones((n, 1))
    feature_mean = np.empty((1,p))
    for i in range(p):
        feature_mean[0,i] = np.mean(feature_df[:,i])
    feature_centered = feature_df - np.dot(ones, feature_mean)
    # get cov matrix # p * p
    cov = np.dot(feature_centered.T, feature_centered) / n
    return cov

def my_SAVE(feature_df_whitened, label_df, flag = "Train", n_directions = 977, verbose = False, **kwargs):

    if flag == "Train":
        # get slices
        label_df.index = np.arange(0, label_df.shape[0], dtype = int)
        slice_index_dict = {1: [], 2: [], 3: [], 4: [], 5: []}
        n = feature_df_whitened.shape[0]
        p = feature_df_whitened.shape[1]
        for i in np.arange(5):
            slice_index_dict[i + 1].extend(label_df[label_df["label"] == (i + 1)].index)
            globals()['feature_df_' + str(i + 1)] = feature_df_whitened[slice_index_dict[i + 1],:]
            globals()["label_df_" + str(i + 1)] = label_df.loc[slice_index_dict[i + 1], "label"]
            # get sliced cov matrices
            globals()['cov_' + str(i + 1)] = get_cov_mat(globals()["feature_df_" + str(i + 1)])
        # concatenate 5 cov matrices, weighted by the proportion of classes
        globals()['weight_df'] = label_df["label"].value_counts() / label_df.shape[0]
        weighted_cov = pd.DataFrame(
            data = np.zeros((globals()['cov_1'].shape[0], globals()['cov_1'].shape[0])), 
            columns = np.arange(p, dtype = int)
        )
        for i in np.arange(5):
            if p == 2:
                if verbose:
                    u, s, vh = np.linalg.svd(globals()['cov_' + str(i + 1)])
                    # u, s = np.linalg.eigh(globals()['cov_' + str(i + 1)])
                    print(f'Eigen values of COV mat: {s}')
                    print(f'Eigen values ratio of COV mat: {s[0] / s[1]:8.4f}')
                    print(f'(1 - Eigen values of COV mat) ratio: {(1 - s[0]) / (1 - s[1]):8.4f}')
                    print(f'(1 - Eigen values of COV mat)^2 ratio: {((1 - s[0]) / (1 - s[1])) ** 2:8.4f}')
                    u, s, vh = np.linalg.svd(np.eye(cov_1.shape[0]) - globals()["cov_" + str(i + 1)])
                    # s, u = np.linalg.eigh(np.eye(cov_1.shape[0]) - globals()["cov_" + str(i + 1)])
                    print(f'Eigen values of (I - COV mat): {s}')
                    print(f'Eigen values ratio of (I - COV mat) : {s[0] / s[1]:8.4f}')
                    print(f'(1 - Eigen values of (I - COV mat)) ratio: {(1 - s[0]) / (1 - s[1]):8.4f}')
                    print(f'(1 - Eigen values of (I - COV mat))^2 ratio: {((1 - s[0]) / (1 - s[1])) ** 2:8.4f}')

            if verbose:
                u, s, vh = np.linalg.svd(np.eye(cov_1.shape[0]) - globals()["cov_" + str(i + 1)])
                # s, u = np.linalg.eigh(np.eye(cov_1.shape[0]) - globals()["cov_" + str(i + 1)])
                print(f'(I - V) eigenvalues: {s}')
                u, s, vh = np.linalg.svd((np.eye(cov_1.shape[0]) - globals()["cov_" + str(i + 1)]) @ (np.eye(cov_1.shape[0]) - globals()["cov_" + str(i + 1)]).T)
                # s, u = np.linalg.eigh((np.eye(cov_1.shape[0]) - globals()["cov_" + str(i + 1)]) @ (np.eye(cov_1.shape[0]) - globals()["cov_" + str(i + 1)]).T)
                print(f'(I - V)^2 eigenvalues: {s}')
            temp = np.eye(cov_1.shape[0]) - globals()["cov_" + str(i + 1)]
            weighted_cov += weight_df.loc[i + 1] * temp @ temp.T
        # u, s, vh = np.linalg.svd(weighted_cov)
        s, u = np.linalg.eigh(weighted_cov)
        b = u[:,:n_directions]
        # transformed to vectors in th original scale
        globals()['directions'] = sigma_pinv_sqrt @ b
        feature_df_reduced = feature_df_whitened @ globals()['directions']
        print()
        print(f'Shape of feature df after post processing: {feature_df_reduced.shape}')  
    elif flag == "Test":
        feature_df_reduced = feature_df_whitened @ globals()['directions']
    return feature_df_reduced, feature_df_whitened, feature_mean, sigma_pinv_sqrt

def sliced_SAVE(feature_df_whitened, label_df, flag = "Train", n_directions = 977, **kwargs):
    if flag == "Train":
        # print('-------------------------------------------------------------------')
        # print(f'Shape of feature df before post processing: {feature_df.shape}')
        globals()['save'] = SlicedAverageVarianceEstimation(n_directions = n_directions, n_slices = 5)
        globals()['save'].fit(feature_df_whitened, label_df)
        globals()['directions'] = globals()['save'].directions_
        feature_df_reduced = globals()['save'].transform(feature_df_whitened)
        # print('-------------------------------------------------------------------')
        # print(f'Shape of feature df after post processing: {feature_df_reduced.shape}') 
    else:
        feature_df_reduced = globals()['save'].transform(feature_df_whitened)
    return feature_df_reduced, feature_df_whitened, feature_mean, sigma_pinv_sqrt

def my_lr(feature_df_train, feature_df_test, label_df_train, label_df_test):
    lr_params_grid = {
        'C': [0, 0.1, 0.5, 1, 10], 
        'tol': [1e-4], 
        'penalty': ['l2'],  
        'random_state': [42], 
        'max_iter': [10000], 
        'verbose': [0], 
        'solver': ['newton-cg'], 
        'n_jobs': [6]
    }

    gs_lr = GridSearchCV(
        estimator = LogisticRegression(), 
        param_grid = lr_params_grid, 
        cv = 5, 
        scoring = 'accuracy', 
        verbose = 0, 
        n_jobs = -1
    )
    gs_lr.fit(feature_df_train, np.array(label_df_train).ravel())
    best_lr = gs_lr.best_estimator_
    lr_pred = best_lr.predict(feature_df_test)
    lr_f1 = f1_score(y_true = label_df_test, y_pred = lr_pred, average = 'macro')
    lr_accu = accuracy_score(y_true = label_df_test, y_pred = lr_pred)
    print('\nBest params: ')
    print(gs_lr.best_params_)
    print(f'Accuracy: {lr_accu:8.4f}\nF1-score: {lr_f1:8.4f}')
    print('-------------------------------------------------------------------')
    return lr_f1, lr_accu

def my_XGB(feature_df_train, feature_df_test, label_df_train, label_df_test):
    xgb_params_grid = {
        'tree_method': ['gpu_hist'], 
        'n_estimators': [50, 100, 200], 
        'max_depth': [3, 4, 5, 6], 
        'n_jobs': [-1], 
        'random_state': [42], 
        'gpu_id': [0], 
        'predictor': ["gpu_predictor"]
    }
    gs = GridSearchCV(
        estimator = XGBClassifier(), 
        param_grid = xgb_params_grid, 
        cv = 5, 
        scoring = 'accuracy', 
        verbose = 0, 
        n_jobs = -1
    )
    gs.fit(feature_df_train, np.array(label_df_train).ravel())
    best_clf = gs.best_estimator_
    pred = best_clf.predict(feature_df_test)
    f1 = f1_score(y_true = label_df_test, y_pred = pred, average = 'macro')
    accu = accuracy_score(y_true = label_df_test, y_pred = pred)
    print('\nBest params: ')
    print(gs.best_params_)
    print(f'Accuracy: {accu:8.4f}\nF1-score: {f1:8.4f}')
    print('----------------------------------------------------------------------------------------------------')
    return f1, accu


def generate_synthetic_data(core_dirs = 3, redundant_dirs = 7, size = 2500, test_size = 0.2, core_distribution = np.random.multivariate_normal, **kwargs): # different dist
    import pandas as pd
    import numpy as np
    import random
    from sklearn.model_selection import train_test_split

    total_features = core_dirs + redundant_dirs
    mean_vec = np.random.random(size = total_features) * 10
    feature_df = core_distribution(size = size, mean = mean_vec, **kwargs) # specify mean + cov
    core_coef = np.random.random(size = core_dirs)
    redundant_coef = np.zeros((redundant_dirs))
    coef = np.concatenate([core_coef, redundant_coef], axis = 0)
    y_train = np.sin(0.2 * np.asarray(feature_df) @ coef) * 10 + np.random.random(size = size)   
    y_train_discrete = pd.cut(y_train, bins = 5, labels = np.arange(5) + 1).astype(int)
    synthetic_feature_df = feature_df
    synthetic_label_df = pd.DataFrame(y_train_discrete, columns = ["label"])
    synthetic_feature_df_train, synthetic_feature_df_test, synthetic_label_df_train, synthetic_label_df_test = train_test_split(
        synthetic_feature_df, synthetic_label_df, 
        test_size = test_size, 
        shuffle = True, 
        random_state = 42
    )
    return synthetic_feature_df_train, synthetic_label_df_train, synthetic_feature_df_test, synthetic_label_df_test

def train_learning_curve(feature_train, label_train, feature_test, label_test, method = my_SAVE, clf = my_lr):
    n_directions_list = list(np.arange(1, feature_train.shape[1] + 1, dtype = int))
    f1_list = []
    accu_list = []
    global feature_mean
    global sigma_pinv_sqrt
    for n_direction in tqdm(n_directions_list):
        globals()['feature_df_reduced_' + str(n_direction) + "_train"], feature_train_whitened, feature_mean, sigma_pinv_sqrt = method(
            feature_train, 
            label_train, 
            flag = "Train", 
            n_directions = n_direction
        )
        globals()['feature_df_reduced_' + str(n_direction) + "_test"], feature_test_whitened, _, _ = method(
            feature_test, 
            label_test, 
            flag = "Test", 
            n_directions = n_direction, 
            feature_mean = feature_mean, 
            sigma_pinv_sqrt = sigma_pinv_sqrt
        )
        curr_test_f1, curr_test_accu = clf(
            feature_df_train = globals()['feature_df_reduced_' + str(n_direction) + "_train"], 
            feature_df_test = globals()['feature_df_reduced_' + str(n_direction) + "_test"], 
            label_df_train = label_train, 
            label_df_test = label_test
        )
        f1_list.append(curr_test_f1)
        accu_list.append(curr_test_accu)

    # train w/ original features -> syncthetic_feature_df -> whitened
    baseline_test_f1, baseline_test_accu = my_lr(
        feature_df_train = feature_train, 
        feature_df_test = feature_test, 
        label_df_train = label_train, 
        label_df_test = label_test
    )
    return f1_list, accu_list, baseline_test_f1, baseline_test_accu, n_directions_list

def train_loop_video(feature_train_std, feature_test_std, label_train, label_test, test_size = 0.2, n_iteration = 1, save = True, suffix = "", method = sliced_SAVE):
    if save:
        if suffix == "":
            raise ValueError("No filename suffix assigned!")
    total_features = feature_train.shape[1]
    columns_list = list(np.arange(1, total_features + 1, dtype = int))
    columns_list.append("baseline")
    f1_df = pd.DataFrame(data = None, columns = columns_list)
    accu_df = pd.DataFrame(data = None, columns = columns_list)
    label_train = np.asarray(label_train, dtype = np.float32).ravel()
    label_test = np.asarray(label_test, dtype = np.float32).ravel()
    for i in tqdm(np.arange(n_iteration)):
        f1_list, accu_list = [], []
        global feature_mean, sigma_pinv_sqrt
        f1_list, accu_list, baseline_f1, baseline_accu, n_directions_list = train_learning_curve(
            feature_train_std, 
            label_train, 
            feature_test_std, 
            label_test, 
            method = method, 
            clf = my_lr
        )
        f1_list.append(baseline_f1)
        accu_list.append(baseline_accu)
        f1_df = pd.concat([f1_df, pd.DataFrame([f1_list], columns = f1_df.columns, index = [i])], axis = 0)
        accu_df = pd.concat([accu_df, pd.DataFrame([accu_list], columns = f1_df.columns, index = [i])], axis = 0)

    if save:
            f1_df.to_csv("../output/f1_df_" + suffix + ".csv", header = True)
            accu_df.to_csv("../output/accu_df_" + suffix + ".csv", header = True)

    return f1_list, accu_list, baseline_f1, baseline_accu, n_directions_list

### Pipeline

In [8]:
feature_df = pd.read_csv(
    "../data/feature_df_951.csv", 
    header = 0,
    index_col = 0
)
temp_label_df = pd.read_csv(
    "../data/label_df_951.csv", 
    header = 0, 
    index_col = 0
)
label_df = pd.Series(data = temp_label_df['0'], index = temp_label_df.index, dtype = np.float32)
feature_df = np.array(feature_df)

pca = PCA(
    n_components = 760 # maximum cols allowed
)
feature_df = pca.fit_transform(feature_df)
'''
train_test_split -> whitening for both parts -> iterations
'''
feature_train, feature_test, label_train, label_test = train_test_split(
    feature_df, label_df, 
    test_size = 0.2, 
    random_state = 42, 
    shuffle = True
)
feature_train_std, feature_mean, sigma_pinv_sqrt = mahalanobis_transformation(
    feature_train, 
    flag = 'Train'
)
feature_test_std, _, _ = mahalanobis_transformation(
    feature_test, 
    flag = 'Test', 
    feature_mean = feature_mean, 
    sigma_pinv_sqrt = sigma_pinv_sqrt
)
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
feature_train_std = std.fit_transform(feature_train_std)
feature_test = std.transform(feature_test_std)

In [9]:
# import warnings
# from sklearn.exceptions import ConvergenceWarning
# warnings.filterwarnings("ignore")

train_loop_video(feature_train_std, feature_test_std, label_train, label_test, test_size = 0.2, n_iteration = 1, save = True, suffix = "video_sliced_ver1", method = sliced_SAVE)

  0%|          | 0/1 [00:00<?, ?it/s]/Users/kangshuoli/miniforge3/envs/env_torch/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 25.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/kangshuoli/miniforge3/envs/env_torch/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/kangshuoli/miniforge3/envs/env_torch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1589, in fit
    fold_coefs_ = Parallel(
  File "/Users/kangshuoli/miniforge3/envs/env_torch/lib


Best params: 
{'C': 0.5, 'max_iter': 10000, 'n_jobs': 6, 'penalty': 'l2', 'random_state': 42, 'solver': 'newton-cg', 'tol': 0.0001, 'verbose': 0}
Accuracy:   0.1885
F1-score:   0.0634
-------------------------------------------------------------------


/Users/kangshuoli/miniforge3/envs/env_torch/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 25.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/kangshuoli/miniforge3/envs/env_torch/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/kangshuoli/miniforge3/envs/env_torch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1589, in fit
    fold_coefs_ = Parallel(
  File "/Users/kangshuoli/miniforge3/envs/env_torch/lib/python3.9/site-packages/joblib/paral


Best params: 
{'C': 1, 'max_iter': 10000, 'n_jobs': 6, 'penalty': 'l2', 'random_state': 42, 'solver': 'newton-cg', 'tol': 0.0001, 'verbose': 0}
Accuracy:   0.1885
F1-score:   0.0634
-------------------------------------------------------------------


/Users/kangshuoli/miniforge3/envs/env_torch/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 25.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/kangshuoli/miniforge3/envs/env_torch/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/kangshuoli/miniforge3/envs/env_torch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1589, in fit
    fold_coefs_ = Parallel(
  File "/Users/kangshuoli/miniforge3/envs/env_torch/lib/python3.9/site-packages/joblib/paral


Best params: 
{'C': 0.5, 'max_iter': 10000, 'n_jobs': 6, 'penalty': 'l2', 'random_state': 42, 'solver': 'newton-cg', 'tol': 0.0001, 'verbose': 0}
Accuracy:   0.1885
F1-score:   0.0634
-------------------------------------------------------------------


  0%|          | 0/1 [00:07<?, ?it/s]


KeyboardInterrupt: 

* Reweight the score w/ confidence
* Transformer w/o last classification layer -> doesn't take time
* NN dimension reduction

